# Projet TAL M1 S1 : POS Tagging

# Implémentation du classifieur

## Import du corpus
Cours Zoom du 17/12 - 35 min

On charge les trois corpus *in-domain* sous la forme de listes de listes de dictionnaires : chaque phrase est une liste de dictionnaire avec une clef : le mot, associée à son POS tag.

In [4]:
def load_corpus(file):
    with open(file, "r") as f:
        content = f.read() # chargement du corpus
    content = content.split("\n\n") # séparation en phrases
    corpus = []
    for phrase in content: # pour chaque phrase
        phrase_list = [] # liste qui contiendra 1 dictionnaire par mot de la phrase
        for line in phrase.splitlines():
            dico_mot = {} # dictionnaire qui contiendra les features d'un mot (mot, lemme, pos)
            if not line.startswith("#"): # on ignore les lignes qui commencent par #
                features = line.split("\t")
                dico_mot[features[1]] = features[3]
                phrase_list.append(dico_mot)
        corpus.append(phrase_list)
    return corpus

gsd_train = load_corpus("corpus-in-domain/fr_gsd-ud-train.conllu")
gsd_test = load_corpus("corpus-in-domain/fr_gsd-ud-test.conllu")
gsd_dev = load_corpus("corpus-in-domain/fr_gsd-ud-dev.conllu")

In [10]:
print("---- Aperçus d'une phrase de chaque corpus-----", end="\n\n")
print(gsd_train[1], end="\n\n")
print(gsd_test[100], end="\n\n")
print(gsd_dev[564])

---- Aperçus d'une phrase de chaque corpus-----

[{"L'": 'DET'}, {'œuvre': 'NOUN'}, {'est': 'AUX'}, {'située': 'VERB'}, {'dans': 'ADP'}, {'la': 'DET'}, {'galerie': 'NOUN'}, {'des': '_'}, {'de': 'ADP'}, {'les': 'DET'}, {'batailles': 'NOUN'}, {',': 'PUNCT'}, {'dans': 'ADP'}, {'le': 'DET'}, {'château': 'NOUN'}, {'de': 'ADP'}, {'Versailles': 'PROPN'}, {'.': 'PUNCT'}]

[{'Ce': 'DET'}, {'résultat': 'NOUN'}, {'provient': 'VERB'}, {'à': 'ADP'}, {'hauteur': 'NOUN'}, {'de': 'ADP'}, {'18,5': 'NUM'}, {'ME': 'NOUN'}, {'de': 'ADP'}, {'la': 'DET'}, {'progression': 'NOUN'}, {'de': 'ADP'}, {'la': 'DET'}, {'rentabilité': 'NOUN'}, {'de': 'ADP'}, {"l'": 'DET'}, {'habitat': 'NOUN'}, {'de': 'ADP'}, {'loisirs': 'NOUN'}, {'qui': 'PRON'}, {'dégage': 'VERB'}, {'une': 'DET'}, {'marge': 'NOUN'}, {'opérationnelle': 'ADJ'}, {'de': 'ADP'}, {'9,5': 'NUM'}, {'%': 'SYM'}, {'contre': 'ADP'}, {'8,5': 'NUM'}, {'%': 'SYM'}, {"l'": 'DET'}, {'année': 'NOUN'}, {'passée': 'ADJ'}, {'.': 'PUNCT'}]

[{'Cette': 'DET'}, {'espèce': 

## Séparation des corpus d'apprentissage et de validation
Cours Zoom du 17/12 - début

Cours Zoom du 19/12 ?

Déjà fait puisqu'on a 3 fichiers différents il me semble.

## Implémentation de l'algorithme de classification
Il faut choisir une régression logistique ou un perceptron moyenné.

## Evaluation des performances sur le corpus de validation

# Evaluation hors-domaine

## Analyse de l'impact du changement de domaine
* identifier causes de la baisse de performance : analyse de sortie, matrice de confusion, erreurs les + fréquentes
* réfléchir à des caractéristiques plus adaptées
* sélection d'un nouvel ensemble d'apprentissage avec des exemples représentatif (généré par un modèle de langue type TP2)

### Corpus UGC (User-generated content)

### Corpus littéraire

## Développement de systèmes robutes au changement de domaine